<a href="https://colab.research.google.com/github/ApplesDaddy/CMPT353Project/blob/main/regression_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!conda --version
!which python
!python --version
!echo $PYTHONPATH

In [ ]:
%env PYTHONPATH=
!echo $PYTHONPATH

In [ ]:
# most recent install link from https://docs.anaconda.com/free/miniconda/miniconda-other-installer-links/
!wget https://repo.anaconda.com/miniconda/Miniconda3-py310_24.1.2-0-Linux-x86_64.sh
!chmod +x Miniconda3-py310_24.1.2-0-Linux-x86_64.sh
!bash ./Miniconda3-py310_24.1.2-0-Linux-x86_64.sh -b -f -p /usr/local/

In [ ]:
!which conda
!conda --version
!conda install --channel defaults conda python=3.10 --yes
!conda update --channel defaults --all --yes

In [ ]:
import sys
sys.path

In [ ]:
!ls /usr/local/lib/python3.10/dist-packages

In [ ]:
sys.path.append("/usr/local/lib/python3.10/site-packages")
!conda config --add channels conda-forge

In [ ]:
!conda create --name musemotion_meta -c conda-forge numpy pandas matplotlib scikit-learn wandb pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

In [ ]:
# !conda init
# !conda activate musemotion_meta
# credit https://stackoverflow.com/a/62668276
%%shell
eval "$(conda shell.bash hook)"
conda activate musemotion_meta

# Load data
_Note_: tempo = bpm

In [ ]:
import urllib.request
urllib.request.urlretrieve("https://github.com/zhandrew4/music-mood-classification/raw/main/muse_v3.csv", "datasets/muse_v3.csv")
urllib.request.urlretrieve("https://github.com/zhandrew4/music-mood-classification/raw/main/full_data.csv", "datasets/full_data.csv")

In [1]:
import pandas as pd
sample = pd.read_csv("datasets/full_data.csv", nrows=3)
sample.columns

Index(['Unnamed: 0', 'track', 'artist', 'seeds', 'spotify_id', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature'],
      dtype='object')

In [2]:
KEEP_COLS_MASTER = ["track", "album", "artist", "release_date", "length",
             "tempo", "key", "time_signature",
             "mood"]

KEEP_COLS_MODEL = ["track", "artist", "tempo", "key", "loudness", "time_signature", "duration_ms"]

# songs_df = pd.read_csv("data_moods.csv", usecols=KEEP_COLS_MASTER)
# make names the indices
songs_df = pd.read_csv("datasets/full_data.csv", usecols=KEEP_COLS_MODEL, parse_dates=True, infer_datetime_format=True)
songs_df.set_index("track", inplace=True)
songs_df

/tmp/ipykernel_72153/842477347.py:9: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  songs_df = pd.read_csv("datasets/full_data.csv", usecols=KEEP_COLS_MODEL, parse_dates=True, infer_datetime_format=True)


,artist,key,loudness,tempo,duration_ms,time_signature
track,,,,,,
'Till I Collapse,Eminem,1,-3.237,171.447,297787,4
St. Anger,Metallica,2,-2.642,185.252,441133,4
Speedin',Rick Ross,1,-4.226,100.059,204960,4
Bamboo Banga,M.I.A.,9,-4.554,125.984,298360,4
Die MF Die,Dope,5,-3.524,126.020,186067,4
...,...,...,...,...,...,...
Secret,Quietdrive,0,-5.126,167.996,258373,4
The Last of the Rest Was the End,Medications,2,-5.774,144.844,324000,3
Lovechild,Daniel Lanois,10,-20.091,79.476,516280,4


Convert dates to numbers, specfically seconds.

_Note_: This goddamn dataset only gives the year for some songs.

In [3]:
# print(songs_df[songs_df["release_date"]=="1993"])
# # credit https://stackoverflow.com/a/54312941
# songs_df["release_date"] = pd.to_datetime(songs_df["release_date"], format="mixed").astype(int)/ 10**9
# songs_df["release_date"]

Load the V, A, D dimension values from the original MuSe dataset and add them to the rest of the data.
^ Join V, A, D columns by key columns title and author.

In [4]:
import pandas as pd

orig_dataset = pd.read_csv("datasets/muse_v3.csv")
orig_dataset.columns
orig_dataset.set_index("track", inplace=True)

In [5]:
VAL_COLS = ["valence_tags","arousal_tags","dominance_tags"]
songs_df = songs_df.join(orig_dataset[VAL_COLS], how="inner")

Model will take data as (`X_train`,`y_train`)
- `X_train` is a 2D NumPy array of shape `n_samples` x `n_features`, i.e. one song per row, one metadata feature per column
- `y_train` is just convert the "mood" column into a 1D NumPy array

In [6]:
# How much should we take for training, testing?
songs_df.index.size

0.7 * songs_df.index.size

77934.5

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

DROP_COLS = VAL_COLS+["artist"]
# use a constant random state for reproducibility
# X = songs_df.drop(columns=DROP_COLS)
# y = songs_df[VAL_COLS]
X_train, X_test, y_train, y_test = train_test_split(songs_df.drop(columns=DROP_COLS), songs_df[VAL_COLS],
                                                    test_size=0.2, random_state=42)

# Model

## Set up Weights & Biases

In [8]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: t0m_x. Use `wandb login --relogin` to force relogin


True

## Multi-Layer Perceptron
[PyTorch tutorial](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

[Building a Regression Model in PyTorch](https://machinelearningmastery.com/building-a-regression-model-in-pytorch/)

[ChatGPT PyTorch Regression Model](https://chat.openai.com/share/8e1237ca-a45f-4abc-8740-25ca00d72bbf)

In [9]:
# from sklearn.neural_network import MLPRegressor

# mlp = MLPRegressor(solver='lbfgs', hidden_layer_sizes=())

# # terrible score lol
# mlp_regression = mlp.fit(X_train, y_train)
# print(mlp_regression.score(X_test, y_test))

In [10]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using {device} device")

Using cuda device


In [12]:
print(X_train)

                 key  loudness    tempo  duration_ms  time_signature
track                                                               
Knockin' On Joe    3   -12.685   80.796       458800               3
Rain               8   -12.726  120.008       364000               4
Twelve             3   -10.527   94.968       247498               4
This Side Of Me    4    -7.971  147.021       251587               4
Love Paradox       7    -5.155  114.994       213608               4
...              ...       ...      ...          ...             ...
Run                9    -6.020   74.537       354547               4
Home               2   -17.867   98.098       195027               4
Peter Street       2    -8.175   83.745        82960               5
Northern Lights    7    -7.469  132.515       197613               4
On the Run         9    -7.162  149.948       195547               4

[89068 rows x 5 columns]


In [13]:
print(X_test)

                                        key  loudness    tempo  duration_ms  \
track                                                                         
Elijah                                    7   -11.552  144.172       317107   
My Saddest Day                            7   -17.936  100.025       153453   
Jehovah's Witness                         1    -2.425  126.997       219307   
Spooky                                    0   -15.713  113.384       164521   
Let's Do Everything for the First Time    9    -6.657  105.457       158200   
...                                     ...       ...      ...          ...   
Anthem                                    7    -7.743  119.949       125470   
Fragile                                   4    -9.535   82.977       260602   
Remember (Christmas)                      4   -19.873  130.113       246333   
Jerusalem                                 3   -10.401   85.663       273640   
Es-So                                     1    -6.31

In [14]:
print(y_test)

                                        valence_tags  arousal_tags  \
track                                                                
Elijah                                      6.825000      5.105000   
My Saddest Day                              4.024000      4.010000   
Jehovah's Witness                           3.745000      5.370000   
Spooky                                      4.400000      6.000000   
Let's Do Everything for the First Time      7.805000      5.440000   
...                                              ...           ...   
Anthem                                      4.873378      3.793108   
Fragile                                     5.326341      3.302073   
Remember (Christmas)                        6.680000      4.370000   
Jerusalem                                   6.260000      4.390000   
Es-So                                       6.430000      4.350000   

                                        dominance_tags  
track                           

In [15]:
print(y_train)

                 valence_tags  arousal_tags  dominance_tags
track                                                      
Knockin' On Joe      3.470000      3.673333        3.166667
Rain                 3.950000      5.570000        4.480000
Twelve               3.150000      3.320000        3.150000
This Side Of Me      6.760000      3.943333        6.326667
Love Paradox         4.000000      5.700000        5.700000
...                       ...           ...             ...
Run                  6.035000      4.680000        5.915000
Home                 6.935000      2.670000        6.100000
Peter Street         8.210000      6.500000        7.210000
Northern Lights      2.053333      3.913333        3.660000
On the Run           5.406667      4.277778        4.994444

[89068 rows x 3 columns]


In [16]:
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_test: (22267, 5)
Shape of y_test: (22267, 3)


In [17]:
# credit: https://stackoverflow.com/questions/71198218/the-simple-mlp-nn-for-regression-in-pytorch-very-slow-learning-rev2

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, dropout_p=0.5):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [18]:
from torch.utils.data import DataLoader, TensorDataset
# convert dataframe to PyTorch Tensor
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).to(device)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False, pin_memory=True)



In [19]:
# input_size = X_train.shape[1]
# hidden_size1 = 6
# hidden_size2 = 4
# output_size = y_train.shape[1]  # Assuming y_train has multiple output columns
# model = NeuralNetwork(input_size, hidden_size1, hidden_size2, output_size).to(device)

hyperparams = dict(
    input_size=X_train.shape[1],
    hidden_size1=5,
    hidden_size2=3,
    output_size=y_train.shape[1],
    dropout_p=0.5,
    lr=0.001,
    epochs=20,
    device=device.type
)

sweep_config = {
    "name": "MLP Sweep",
    "method": "grid",
    "metric": {"goal": "minimize", "name": "test loss"},
    "parameters": {
        "hidden_size1": {"values": list(range(3,9))},
        "hidden_size2": {"values": list(range(3,9))},
        "epochs": {"values": [5, 10, 15]},
        "dropout_p": {"values": [0.3, 0.5, 0.7]},
    }
}

In [20]:
sweep_id = wandb.sweep(sweep_config, project="Music Mood Prediction")

Create sweep with ID: 45y17bhm
Sweep URL: https://wandb.ai/t0m_x/Music%20Mood%20Prediction/sweeps/45y17bhm


In [ ]:
import torch.optim as optim
import torch.utils
import torch.utils.data

# Build and train a model
def pipeline(config=None, train_loader: torch.utils.data.DataLoader=train_loader) -> nn.Module:
    # Initialize a new wandb run for this hyperparameter configuration
    # config will be set by Sweep Controller
    wandb.init(project="Music Mood Prediction", name="MLP Regressor", config=config)
    config = wandb.config

    # build the model
    model = NeuralNetwork(hyperparams["input_size"], hyperparams["hidden_size1"], hyperparams["hidden_size2"], hyperparams["output_size"], hyperparams["dropout_p"]).to(device)

    # define the loss function and the optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])

    wandb.watch(model, criterion, log="all", log_freq=10)

    num_epochs = hyperparams["epochs"]
    avg_loss = 0.0
    for epoch in range(num_epochs):
        avg_loss = train_epoch(train_loader, model, optimizer, criterion)
        wandb.log({"epoch": epoch, "train loss": avg_loss})
    
    return model
        
def train_epoch(train_loader, model, optimizer, criterion):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        # forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

# model = pipeline()
wandb.agent(sweep_id, function=pipeline, count=6)

In [22]:
def test(test_loader: torch.utils.data.DataLoader, model: torch.nn.Module, criterion: torch.nn.Module,
         log_table: wandb.Table):
    """record test examples and predictions in W&B table"""
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item() * inputs.size(0)

            # Display and log some test examples <- input and output
            print(f'input: {inputs[0].cpu().numpy()}')
            print(f'output: {outputs[0].cpu().numpy()}')
            log_table.add_data(

            )

        test_loss /= len(test_loader.dataset)
        wandb.log({"test loss": test_loss})
        print(f'test loss: {test_loss:.4f}')

test_table = wandb.Table(columns=[
    "song name",
    "artist",
    "tempo",
    "key signature",
    "loudness",
    "time signature",
    "duration (ms)",
    "predicted valence",
    "predicted arousal",
    "predicted dominance"
])
test(test_loader, model, criterion, test_table)
wandb.log({"test examples": test_table})

NameError: name 'model' is not defined

In [ ]:
# sweep_id = wandb.sweep(sweep_config, project="Music Mood Prediction")
# wandb.agent(sweep_id, function=pipeline)

In [ ]:
wandb.finish()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f75d9cb5b80>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f75cdfd0350, raw_cell="wandb.finish()" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Basb9804u-b07.csil.sfu.ca/usr/shared/CMPT/scratch/wxa38/CMPT353Project/regression_Colab.ipynb#Y304sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f75d9cb5b80>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f75cdfd1940, execution_count=25 error_before_exec=None error_in_exec=[Errno 32] Broken pipe info=<ExecutionInfo object at 7f75cdfd0350, raw_cell="wandb.finish()" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Basb9804u-b07.csil.sfu.ca/usr/shared/CMPT/scratch/wxa38/CMPT353Project/regression_Colab.ipynb#Y304sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

## Histogram-Based Gradient Boosted Tree
- [scikit-learn tutorial](https://scikit-learn.org/stable/modules/ensemble.html#histogram-based-gradient-boosting)
- [regressor API reference](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html#sklearn.ensemble.HistGradientBoostingRegressor)

In [ ]:
wandb.init(
    project="Music Mood Prediction",
    name="Histogram-based Gradient Boosting Regressor"
    )

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_validate

hgbt = MultiOutputRegressor(HistGradientBoostingRegressor(random_state=0))
regression = hgbt.fit(X_train, y_train)

In [ ]:
r2 = regression.score(X_test, y_test)
wandb.log({"Coefficient of Correlation (R^2)": r2})
# The outliers chart only supports univariate (i.e. 1D output) regression
# wandb.sklearn.plot_regressor(regression,
#                             X_train, X_test,
#                             y_train, y_test,
#                             model_name="hist_gradient_boosting_regressor")
wandb.sklearn.plot_learning_curve(regression, X_train, y_train)
wandb.sklearn.plot_summary_metrics(regression, X_train, y_train, X_test, y_test)

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold

param_grid = {
    "learning_rate": [0.1, 0.01, 0.001],
    "n_estimators": [100, 200, 300],
    "max_depth": [3, 5, 7],
    "min_samples_split": [20, 35, 50],
    "min_samples_leaf": list(range(1,6))
}

cv_fold = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(hgbt, param_grid, cv=cv_fold, n_jobs=-1, scoring="neg_mean_squared_error")
param_search_results = grid_search.fit(X_train, y_train)

best_model = param_search_results.best_estimator_
best_params = param_search_results.best_params_
wandb.log({"Coefficient of Correlation (R^2)": best_model.score(X_test, y_test)})

In [ ]:
wandb.sklearn.plot_learning_curve(best_model, X_train, y_train)
wandb.sklearn.plot_summary_metrics(best_model, X_train, y_train, X_test, y_test)

In [ ]:
wandb.finish()

In [ ]:
regression.score(X_test, y_test)